In [9]:
arr = [5, 3, 6, 1, 99]

<h4>Selection sort</h4>

In [ ]:
import Data.List

In [ ]:
ssort :: Ord a => [a] -> [a]
ssort [] = []
ssort xs = m : ssort (delete m xs)
            where m = minimum xs

In [ ]:
split :: Ord a => [a] -> a -> [a] -> [a]
split [] m r = m : (ssort' r)
split (x:xs) m r = if x < m
                     then split xs x (m:r)
                   else split xs m (x:r)
                   
ssort' []     = []
ssort' (x:xs) = split xs x []

In [ ]:
ssort arr
ssort' arr

<h4>Insertion sort</h4>

In [ ]:
insert :: Ord a => a -> [a] -> [a]
insert x xs = takeWhile ((<=) x) xs ++ [x] ++ dropWhile ((<=) x) xs

isort :: Ord a => [a] -> [a]
isort []     = []
isort (x:xs) = insert x (isort xs)

isort' xs = foldr insert [] xs

In [ ]:
insert' key []       = [key]
insert' key l@(x:xs)
    | key <= x       = key : l
    | otherwise      = x : (insert' key xs)
    
isort'' xs = foldr insert' [] xs

In [ ]:
reverse $ isort arr
reverse $ isort' arr
isort'' arr

<h4>Quick sort</h4>

In [ ]:
qsort :: Ord a => [a] -> [a]
qsort []           = []
qsort (pivot:rest) = qsort lower ++ [pivot] ++ qsort upper
                        where lower = [x | x <- rest, x <= pivot]
                              upper = [x | x <- rest, x > pivot]

In [ ]:
qsort' :: Ord a => [a] -> [a] -> [a]
qsort' [] s = s
qsort' (pivot:rest) s =
    qsort' lower (pivot : (qsort' upper s))
    where lower = [x | x <- rest, x <= pivot]
          upper = [x | x <- rest, x > pivot]  

In [ ]:
qsort arr
qsort' arr []

<h4>Merge sort</h4>

In [1]:
import Control.Monad.Writer
import Control.Monad.Identity

In [2]:
type ArrayWithLogging a = WriterT [String] Identity [a]

In [6]:
{-# LANGUAGE FlexibleContexts #-}

merge :: (Show a, Ord a) => [a] -> [a] -> ArrayWithLogging a
merge a        []     = tell ["Returning " ++ show a] >> return a
merge []       b      = tell ["Returning " ++ show b] >> return b
merge a@(x:xs) b@(y:ys)
  | x <= y    = message x xs b  >> fmap (x:) (merge xs b)
  | otherwise = message y a  ys >> fmap (y:) (merge a ys)
  where
    message e ls rs = tell ["Min " ++ show e ++ " merging " ++ show ls ++ " with " ++ show rs]

merge' :: (Show a, Ord a) => ArrayWithLogging a-> ArrayWithLogging a-> ArrayWithLogging a
merge' a b = do
   a1 <- a
   b1 <- b
   merge a1 b1

In [12]:
msort :: (Show a, Ord a) => [a] -> ArrayWithLogging a
msort [] = tell ["Empty base case"] >> return []
msort [x] = tell ["Single element base case : " ++ show x] >> return [x]
msort xs = do
    tell ["Split 1 : " ++ show xs1 ++ " . Split 2 : " ++ show xs2 ++ " ."]
        >> merge' (msort xs1) (msort xs2)
    where
        xs1 = take k xs
        xs2 = drop k xs
        k = (length xs) `div` 2

In [13]:
msort arr

WriterT (Identity ([1,3,5,6,99],["Split 1 : [5,3] . Split 2 : [6,1,99] .","Split 1 : [5] . Split 2 : [3] .","Single element base case : 5","Single element base case : 3","Min 3 merging [5] with []","Returning [5]","Split 1 : [6] . Split 2 : [1,99] .","Single element base case : 6","Split 1 : [1] . Split 2 : [99] .","Single element base case : 1","Single element base case : 99","Min 1 merging [] with [99]","Returning [99]","Min 1 merging [6] with [99]","Min 6 merging [] with [99]","Returning [99]","Min 1 merging [3,5] with [6,99]","Min 3 merging [5] with [6,99]","Min 5 merging [] with [6,99]","Returning [6,99]"]))

<h5>Bottom-up merge sort</h5>

In [21]:
split :: (Show a, Ord a) => [a] -> [[a]]
split []     = []
split (x:xs) = [x] : split xs

mergePairs :: (Show a, Ord a) => [[a]] -> [[a]]
mergePairs []           = []
mergePairs x@[l]        = x
mergePairs (l1:l2:rest) = (fst $ runIdentity $ runWriterT (merge l1 l2)) : (mergePairs rest)

In [22]:
msort'' :: (Show a, Ord a) => [a] -> [a]
msort'' l = ms (split l)
    where ms [r] = r
          ms l = ms (mergePairs l)

In [23]:
msort'' arr

[1,3,5,6,99]